# Warehouse Simulation Based on SimPy
This notebook provides the code for simulating warehouse activity using several algorithm to pick the order

## Flow
The flow of this simulation code is as follow:
1. prepare required module
1. define combination of rules for order picking
1. prepare SimPy environment
1. run simulation
1. generate report

### 1. Prepare required module

In [1]:
import simpy
import pandas as pd
import random
from datetime import timedelta
from modules import general_function as gf
from modules import pooling_triggers
from modules import batchings
from modules import routings

### 2. Define rules combination

| Option    | Trigger   | Batching  | Routing       | Cart  |
| ---       | ---       | ---       | ---           | ---   |
| 1         | FTWB      | FCFS      | S-Shape       | 50    |
| 2         | VTWB      | Seed      | Largest Gap   | 100   |
| 3         | Max Picker |          |               | 200   |
| 4         | Max Cart
| 5         | Urgent First + Max Picker
| 6         | Urgent First + Max Cart


In [2]:
trigger_opt = 1
batching_opt = 1
routing_opt = 1
picker_num = 1
cart_opt = 1

### 3. Prepare SimPy environment

In [8]:
# Reading order file
fname = gf.reading_file()

# Processing file
# Making Index
a = len(fname) + 1
orderFile = list(range(1, a)) + ['Avg']
# Counting total order
totalOrder = gf.count_total_order(fname)
# Counting total item picked
totalItemPicked = gf.count_total_item(fname)

# Prepare report container
totalCompletionTime = list()
averagePickerUtility = list()
averageCartUtility = list()
onTimeDelivery = list()

In [4]:
# Assign cart capacity
cart_capacity = 0
if cart_opt == 1:
    cart_capacity = 50
elif cart_opt == 2:
    cart_capacity = 100
elif cart_opt == 3:
    cart_capacity = 200

### 4. Run Simulation

In [5]:
for fn in fname:
    env = simpy.Environment()
    # Setting delta for FTWB
    delta = timedelta(minutes=12).seconds

    # Determine batching method
    batching = batchings.batchings(batching_opt, cart_capacity)
    # Determine routing method
    routing = routings.routings(batching_opt)
    # Prepare pooling and determine trigger method
    pool_trigger = pooling_triggers.pooling_triggers(trigger_opt, env, picker_num, batching, routing, cart_capacity, delta)

    # Prepare env processing
    env.process(pool_trigger.run_simpy(fn))

    # Define simulation time
    limit = timedelta(hours=9).seconds

    # Run simulation
    env.run(until=limit)

    # Listing total completion time
    totalCompletionTime.append(pool_trigger.completionTime)
    # Counting and listing average picker utility
    avePickerUtility = round(pool_trigger.completionTime/timedelta(hours=8), 2)
    averagePickerUtility.append(avePickerUtility)
    # Counting & listing average cart utility
    aveCartUtility = round(pool_trigger.cartUtility/pool_trigger.fileCount, 2)
    averageCartUtility.append(aveCartUtility)
    # Listing total on time delivery
    onTimeDelivery.append(pool_trigger.onTime)

In [6]:
# Counting average of total completion time
totalCompletionTime = gf.average_tct(totalCompletionTime)
# Counting average of average picker utility
averagePickerUtility = gf.count_average(averagePickerUtility)
# Counting average of average cart utility
averageCartUtility = gf.count_average(averageCartUtility)
# Counting average of on time delivery
onTimeDelivery = gf.count_average(onTimeDelivery)

### 5. Generate Report

In [7]:
# Put result on file
result = pd.DataFrame({
    'Order File': orderFile,
    'Total Order': totalOrder,
    'Total Completion Time': totalCompletionTime,
    'Total Item Picked': totalItemPicked,
    'Average Picker Utility': averagePickerUtility,
    'Average Cart Utility': averageCartUtility,
    'On Time Delivery': onTimeDelivery
    }, columns = ['Order File', 'Total Order', 'Total Completion Time', 'Total Item Picked', 'Average Picker Utility', 'Average Cart Utility', 'On Time Delivery'])
filename = str(trigger_opt) + str(batching_opt) + str(routing_opt) + str(picker_num) + str(cart_opt) + '.csv'
result.to_csv('result/simpy' + filename, index = False)
print(filename)

11111.csv
